In [4]:
%pip install sentence-transformers

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, -1, Finished, Available)


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: /nfs4/pyenv-ad468f31-aee7-4654-8a98-8f16dc943147/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import os

DATA_DIR = "/lakehouse/default/Files/mladsdata/"

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 17, Finished, Available)

In [6]:
os.listdir(DATA_DIR)

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 18, Finished, Available)

['ai_Posts.csv',
 'ai_posts_aoai_4.parquet',
 'dolly_file.parquet',
 'finetuning',
 'gender_bias',
 'generic_occupation_stereotype.xlsx',
 'occupation_translations.parquet',
 'rag',
 'recommenders',
 'structured_stereotypes.xlsx']

# Co-occurrence of bias adjectives across occupations

In [7]:
structured_stereotypes = pd.read_excel(DATA_DIR + "structured_stereotypes.xlsx", index_col=0)
structured_stereotypes

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 19, Finished, Available)

,id,stereotype,bias_adjective,bias_type,bias_sentiment,occupation,batch_id
0,1,Accountants are boring.,boring,personal trait,-1,Accountants,0
1,2,All accountants are nerds.,nerds,personal trait,-1,Accountants,0
2,3,Accountants are good with money.,good,personal trait,1,Accountants,0
3,4,Accountants are smart.,smart,personal trait,1,Accountants,0
4,5,Accountants are detail oriented.,detail oriented,personal trait,1,Accountants,0
...,...,...,...,...,...,...,...
11202,16,Waiters always get your drink order wrong.,incompetent,service,-1,Waiters,588
11203,17,Waiters always clear your plates as soon as yo...,efficient,service,1,Waiters,588
11204,18,Waiters always check on you to see how your me...,attentive,service,1,Waiters,588
11205,19,Waiters always are upselling you on dessert.,greedy,sales,-1,Waiters,588


In [8]:
basket_item = pd.melt(structured_stereotypes[ ['occupation', 'bias_adjective'] ].reset_index().rename(columns={'index':'basket'}), 
                        id_vars='basket', 
                        value_vars=['occupation', 'bias_adjective'],
                        var_name='category', value_name='item'
                     ).sort_values(['basket', 'category'])

basket_item

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 20, Finished, Available)

,basket,category,item
11207,0,bias_adjective,boring
0,0,occupation,Accountants
11208,1,bias_adjective,nerds
1,1,occupation,Accountants
11209,2,bias_adjective,good
...,...,...,...
11204,11204,occupation,Waiters
22412,11205,bias_adjective,greedy
11205,11205,occupation,Waiters
22413,11206,bias_adjective,greedy


In [9]:
def get_item_pair_stats(item_pair_df):
    # item_pair_df must have columns named 'basket', and 'item'.
    
    import sqlite3 # we're using this for portability; you can run this code on your favorite SQL database.

    db = sqlite3.connect(":memory:")
    
    item_pair_df.to_sql("basket_item", db, if_exists="replace")


    ITEM_PAIR_STATS_QUERY = """with 
      bi as (
        select basket, item
          from basket_item
          group by basket, item  -- be sure we only count one of each kind of item per basket
      ),
      item_counts as (
        select item, count(*) item_count -- same as the number of baskets containing this item (see above)
          from bi
          group by item
      ),
      bi_count as (
        select bi.*, ic.item_count  -- basket, item, item_count
          from bi
            join item_counts ic on bi.item=ic.item
      ),
      ips as (
          select bi1.item item1, bi2.item item2,
                  bi1.item_count item1_count, bi2.item_count item2_count,
                  count(*) as both_count              
              from bi_count bi1
                join bi_count bi2  -- joining the table to itself
                  on bi1.basket = bi2.basket  -- two items in the same basket
                  and bi1.item != bi2.item    -- don't count the item being in the basket with itself
              group by bi1.item, bi1.item_count, 
                       bi2.item, bi2.item_count
      ),
      cc as (
        SELECT item1, item2, item1_count, item2_count, both_count,
              CAST(item1_count AS FLOAT)/(select count(distinct basket) from basket_item) as item1_prevalence, -- fraction of all baskets with item1
              CAST(item2_count AS FLOAT)/(select count(distinct basket) from basket_item) as item2_prevalence, -- fraction of all baskets with item2
              CAST(both_count AS FLOAT)/CAST(item1_count AS FLOAT) AS confidence  -- fraction of baskets with item1 that also have item2
          FROM ips
      )
    select *, confidence/item2_prevalence lift from cc
    """

    return pd.read_sql_query(ITEM_PAIR_STATS_QUERY, db)


def get_nodes_and_edges_from_item_pair_stats(cooccurrence_pdf):
    """
    Convert a Pandas dataframe of item-pair statistics to separate dataframes for nodes and edges.
    """
    from collections import Counter
    
    item_stats = {r['item1']:{'count':r['item1_count'], 'prevalence':r['item1_prevalence']} 
                    for idx, r in cooccurrence_pdf.iterrows()}
 
    item_stats.update({r['item2']:{'count':r['item2_count'], 'prevalence':r['item2_prevalence']} 
                    for idx, r in cooccurrence_pdf.iterrows()})
 
    nodes_df = pd.DataFrame([{'label':k,'count':v['count'], 'prevalence':v['prevalence']}  
                    for k,v in item_stats.items()])
    nodes_df['id'] = nodes_df.index
   
    edges_df = cooccurrence_pdf.copy()
    node_id = {r['label']:r['id'] for idx, r in nodes_df.iterrows()}
    edges_df['from'] = [node_id[nn] for nn in edges_df['item1']]
    edges_df['to'] = [node_id[nn] for nn in edges_df['item2']]
    
    print("Your graph will have {0} nodes and {1} edges.".format( len(nodes_df), len(edges_df) ))
 
    return nodes_df, edges_df[[ 'from', 'to', 'both_count', 'confidence', 'lift']]


def get_vis_js_html(nodes_df, edges_df):
    """
    Generate HTML encoding vis_js graph from Pandas dataframes of nodes and edges.
    """
    nodes_str = nodes_df.to_json(orient='records')
    edges_str = edges_df.to_json(orient='records')
    
    max_weight = max(edges_df['weight'])

    html_string = ( 
    f'    <script type="text/javascript">NODE_LIST={nodes_str};FULL_EDGE_LIST={edges_str};</script>\n'
    '\n'
        '\n'
    '        <script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>\n'
    '        <script type="text/javascript">\n'
    '            function hello(){console.info("hello")}\n'
    '\n'
    '            const sign_color = {pos:"blue", neg:"red", zero:"black"}\n'
    '            const options = {physics:{maxVelocity: 1, minVelocity: 0.01}}\n'
    '            var edgeFilterSlider\n'
    '            var mynetwork\n'
    '            var motion_flag = false\n'
    '            function toggle_motion(){\n'
    '                motion_flag = !motion_flag\n'
    '                mynetwork.setOptions( { physics: motion_flag } )\n'
    '            }\n'
    '\n'
    '            function edgesFilter(edge){\n'
    '                return edge.value >= edgeFilterSlider.value\n'
    '            }\n'
    '\n'
    '            function init_network(){\n'
    '                document.getElementById("min_edge_weight_display").value = 0.5\n'
    '                document.getElementById("min_edge_weight").onchange = function(){\n'
    '                    document.getElementById("min_edge_weight_display").value = this.value\n'
    '                }\n'
    '\n'
    '                edgeFilterSlider = document.getElementById("min_edge_weight")\n'
    '                edgeFilterSlider.addEventListener("change", (e) => {edgesView.refresh()})\n'
    '                var container = document.getElementById("mynetwork")\n'
    '                var EDGE_LIST = []\n'
    '                for (var i = 0; i < FULL_EDGE_LIST.length; i++) {\n'
    '                    var edge = FULL_EDGE_LIST[i]\n'
    '                    edge["value"] = Math.abs(edge["weight"])\n'
    '                    edge["title"] = "weight " + edge["weight"]\n'
    '                    edge["sign"] = (edge["weight"] < 0) ? "neg" : "pos";\n'
    '                    edge["color"] = {color: sign_color[edge["sign"]] };\n'
    '                    edge["arrows"] = "to"\n'
    '                    EDGE_LIST.push(edge)\n'
    '                }\n'
    '\n'
    '                var nodes = new vis.DataSet(NODE_LIST)\n'
    '                var edges = new vis.DataSet(EDGE_LIST)\n'
    '                var nodesView = new vis.DataView(nodes)\n'
    '                var edgesView = new vis.DataView(edges, { filter: edgesFilter })\n'
    '                var data = { nodes: nodesView, edges: edgesView }\n'
    '                mynetwork = new vis.Network(container, data, options)\n'
    '\n'
    '            }\n'
    '            init_network()\n'
    '        </script>\n'
    '        <style type="text/css">#mynetwork {width: 100%; height: 500px; border: 3px}</style>\n'
    '        <button onclick=toggle_motion()>Toggle motion</button>\n'
    '        <div class="slidercontainer">\n'
    '            <label>minimum edge weight:\n'
    f'                <input type="range" min="0" max="{max_weight}" value="{max_weight/2}" step="{max_weight/100}" class="slider" id="min_edge_weight">\n'
    '                <input type="text" id="min_edge_weight_display" size="2">\n'
    '            </label>\n'
    '        </div>\n'
    '        <div id="mynetwork"></div>\n'

    )
    return html_string
        
    
def export_to_vis_js(nodes_df, edges_df, title, html_file_name):
    """
    Generate vis_js graph from Pandas dataframes of nodes and edges, and write to HTML file.
    """
    
    vis_js_html = get_vis_js_html(nodes_df, edges_df)
    page_html =  ('<!DOCTYPE html>\n'
        '<html lang="en">\n'
        '    <head>\n'
        f'       <title>{title}</title>\n'
        '    </head>\n'
        '    <body onload=init_network()>\n'
        f'{vis_js_html}'
        '\n'
        '    </body>\n'
        '</html>\n')
    
    with open(html_file_name, "wt") as html_file: 
        html_file.write(page_html)

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 21, Finished, Available)

In [10]:
item_pair_stats = get_item_pair_stats(basket_item)

item_pair_stats

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 22, Finished, Available)

,item1,item2,item1_count,item2_count,both_count,item1_prevalence,item2_prevalence,confidence,lift
0,-,Construction Workers,1,409,1,0.000089,0.036495,1.000000,27.400978
1,Accountants,affluent,280,1,1,0.024984,0.000089,0.003571,40.025000
2,Accountants,aggressive,280,73,1,0.024984,0.006514,0.003571,0.548288
3,Accountants,always,280,85,2,0.024984,0.007585,0.007143,0.941765
4,Accountants,always calculating,280,2,2,0.024984,0.000178,0.007143,40.025000
...,...,...,...,...,...,...,...,...,...
10159,young,Teachers,6,298,2,0.000535,0.026591,0.333333,12.535794
10160,younger,Programmers,2,181,1,0.000178,0.016151,0.500000,30.958564
10161,younger,Salespeople,2,438,1,0.000178,0.039083,0.500000,12.793379
10162,your best interest,Doctors,1,409,1,0.000089,0.036495,1.000000,27.400978


In [11]:
nodes, edges = get_nodes_and_edges_from_item_pair_stats( item_pair_stats[ item_pair_stats['both_count'] > 5 ] )

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 23, Finished, Available)

Your graph will have 214 nodes and 734 edges.


In [12]:
occupations = set(structured_stereotypes['occupation'])

nodes['category'] = [ 'occupation' if x in occupations else 'adjective' for x in nodes['label'] ]
nodes['shape'] = [ 'box' if cat == 'occupation' else 'ellipse' for cat in nodes['category'] ]
nodes['color'] = [ '#FF9999' if cat == 'occupation' else '#9999FF' for cat in nodes['category'] ]

def make_node_title(row, sentence_df):
    title = f"{row['label']}\n({row['category']}, {row['count']} examples)"
    if row['category'] == 'adjective':
        adjective = row['label']
        examples = sentence_df[ sentence_df['bias_adjective'] == adjective ]['stereotype'].values
        top_examples = [ f"{x[0]} ({x[1]} times)" for x in Counter(examples).most_common(5) ]
        title += '\n' + '\n'.join(top_examples)
        
    return title

nodes['title'] = [ make_node_title(row, structured_stereotypes) for row in nodes.to_dict(orient='records') ]

nodes

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 24, Finished, Available)

,label,count,prevalence,id,category,shape,color,title
0,Accountants,280,0.024984,0,occupation,box,#FF9999,"Accountants\n(occupation, 280 examples)"
1,Actors,310,0.027661,1,occupation,box,#FF9999,"Actors\n(occupation, 310 examples)"
2,Architects,406,0.036227,2,occupation,box,#FF9999,"Architects\n(occupation, 406 examples)"
3,Artists,516,0.046043,3,occupation,box,#FF9999,"Artists\n(occupation, 516 examples)"
4,Bankers,476,0.042473,4,occupation,box,#FF9999,"Bankers\n(occupation, 476 examples)"
...,...,...,...,...,...,...,...,...
209,wealthy,26,0.002320,209,adjective,ellipse,#9999FF,"wealthy\n(adjective, 26 examples)\nDoctors mak..."
210,well-dressed,15,0.001338,210,adjective,ellipse,#9999FF,"well-dressed\n(adjective, 15 examples)\nRecept..."
211,wholesome,11,0.000982,211,adjective,ellipse,#9999FF,"wholesome\n(adjective, 11 examples)\nFarmers a..."
212,work-focused,20,0.001785,212,adjective,ellipse,#9999FF,"work-focused\n(adjective, 20 examples)\nEngine..."


In [13]:
edges['weight'] = edges['confidence']

edges

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 25, Finished, Available)

,from,to,both_count,confidence,lift,weight
14,0,49,19,0.067857,10.562153,0.067857
31,0,67,12,0.042857,10.219149,0.042857
52,0,102,9,0.032143,3.398349,0.032143
57,0,108,8,0.028571,8.210256,0.028571
59,0,110,7,0.025000,6.090761,0.025000
...,...,...,...,...,...,...
10012,209,9,15,0.576923,15.808257,0.576923
10037,210,24,7,0.466667,13.909397,0.466667
10070,211,12,11,1.000000,22.192079,1.000000
10121,212,11,20,1.000000,31.044321,1.000000


In [14]:
vis_js_html = get_vis_js_html(nodes, edges)
displayHTML(vis_js_html)

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 26, Finished, Available)

# Structuring data by clustering

## Semantic clustering of generic sterotype sentences

In [15]:
generic_occupation_stereotype = pd.read_excel(DATA_DIR + "generic_occupation_stereotype.xlsx", index_col=0)

generic_occupation_stereotype.head()

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 27, Finished, Available)

,stereotype,generic,occupation
0,CEOs are always looking for ways to make more ...,They are always looking for ways to make more ...,CEOs
1,CEOs are power-hungry and only care about them...,They are power-hungry and only care about them...,CEOs
2,CEOs are always working and never have any tim...,They are always working and never have any tim...,CEOs
3,CEOs are unapproachable and think they are bet...,They are unapproachable and think they are bet...,CEOs
4,"CEOs are cold and calculating, and only care a...","They are cold and calculating, and only care a...",CEOs


In [16]:
from sentence_transformers import SentenceTransformer
sentxformer = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

# read about models here: https://www.sbert.net/docs/pretrained_models.html
# paraphrase-MiniLM-L6-v2 all-mpnet-base-v2 all-MiniLM-L12-v2
generic_occupation_stereotype['embedding'] = sentxformer.encode(generic_occupation_stereotype['generic'].values).tolist()

def add_cluster_cols(df, embedding_col='embedding', letters='ABCDE', max_threshold=1):
    from scipy.cluster.hierarchy import ward, fcluster
    from scipy.spatial.distance import pdist
    import math

    # cluster the sentence vectors at various levels
    X = df[embedding_col].tolist()
    y = pdist(X, metric='cosine')
    z = ward(y)

    for i in range(len(letters)):
        letter = letters[i]
        col_name = f'cluster_{letter}'
        cluster_id = fcluster(z, max_threshold/2**i, criterion='distance')
        digits = 1 + math.floor(math.log10(max(cluster_id)))
        df[col_name] = [letter + str(cid).zfill(digits) for cid in cluster_id]

    cluster_cols = [c for c in df.columns if c.startswith('cluster_')]
    return df.sort_values(by=cluster_cols)


generic_occupation_stereotype = add_cluster_cols(generic_occupation_stereotype, max_threshold=4)

generic_occupation_stereotype

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 28, Finished, Available)

,stereotype,generic,occupation,embedding,cluster_A,cluster_B,cluster_C,cluster_D,cluster_E
351,All bankers are rich.,They are rich.,Bankers,"[0.22004050016403198, 0.43527528643608093, -0....",A01,B01,C001,D001,E0001
413,Doctors are rich.,They are rich.,Doctors,"[0.22004050016403198, 0.43527528643608093, -0....",A01,B01,C001,D001,E0001
556,All physicists are rich.,They are rich.,Physicists,"[0.22004039585590363, 0.4352751672267914, -0.4...",A01,B01,C001,D001,E0001
925,Bankers are rich and have a lot of money.,They are rich and have a lot of money.,Bankers,"[0.2578396797180176, 0.10808152705430984, -0.4...",A01,B01,C001,D001,E0001
975,Doctors are rich.,They are rich.,Doctors,"[0.22004050016403198, 0.43527528643608093, -0....",A01,B01,C001,D001,E0001
...,...,...,...,...,...,...,...,...,...
2521,Salespeople are go-getters.,They are go-getters.,Salespeople,"[0.49709993600845337, -0.06929469108581543, -0...",A17,B68,C243,D669,E1285
2532,Salespeople are go-getters,They are go-getters,Salespeople,"[0.4964907765388489, -0.133231058716774, -0.31...",A17,B68,C243,D669,E1285
144,Lawyers are always out to get you.,They are always out to get you.,Lawyers,"[0.3699645400047302, -0.04968713968992233, 0.0...",A17,B68,C243,D670,E1286
3145,Police Officers are always out to get you.,They are always out to get you.,Police Officers,"[0.3699645400047302, -0.04968713968992233, 0.0...",A17,B68,C243,D670,E1286


StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 41, Finished, Available)

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 42, Finished, Available)

In [17]:
generic_occupation_stereotype[ generic_occupation_stereotype['cluster_B'] == 'B29'] # 'dull' and 'boring'
# generic_occupation_stereotype[ generic_occupation_stereotype['cluster_B'] == 'B27']  # 'dishonest' and 'liars'

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 29, Finished, Available)

,stereotype,generic,occupation,embedding,cluster_A,cluster_B,cluster_C,cluster_D,cluster_E
20,All accountants are dull.,They are dull.,Accountants,"[-0.1978253871202469, 0.044321268796920776, 0....",A07,B29,C093,D242,E0448
1016,Accountants are dull.,They are dull.,Accountants,"[-0.1978253871202469, 0.044321268796920776, 0....",A07,B29,C093,D242,E0448
2042,Accountants are very dull.,They are very dull.,Accountants,"[-0.1891990453004837, 0.1373164802789688, 0.54...",A07,B29,C093,D242,E0448
2954,Bankers are dull.,They are dull.,Bankers,"[-0.19782574474811554, 0.04432123899459839, 0....",A07,B29,C093,D242,E0448
930,Bankers are boring and have no sense of humor.,They are boring and have no sense of humor.,Bankers,"[0.14831902086734772, -0.28764215111732483, 0....",A07,B29,C093,D243,E0449
1128,Bankers are boring and have no sense of humor.,They are boring and have no sense of humor.,Bankers,"[0.14831902086734772, -0.28764215111732483, 0....",A07,B29,C093,D243,E0449
2989,Engineers are boring and have no sense of style.,They are boring and have no sense of style.,Engineers,"[-0.08724543452262878, -0.13345880806446075, 0...",A07,B29,C093,D243,E0449
5,All accountants are boring.,They are boring.,Accountants,"[0.085758738219738, -0.33333638310432434, 0.54...",A07,B29,C093,D243,E0450
491,Lawyers are boring.,They are boring.,Lawyers,"[0.08575821667909622, -0.33333620429039, 0.546...",A07,B29,C093,D243,E0450
626,Teachers are boring and unimaginative.,They are boring and unimaginative.,Teachers,"[0.030932055786252022, -0.31621211767196655, 0...",A07,B29,C093,D243,E0450


In [18]:
cluster_level = 'cluster_B'
top5_dict = {}

for cluster_id in sorted(list(set(generic_occupation_stereotype[cluster_level]))):
    top5 = Counter(generic_occupation_stereotype[generic_occupation_stereotype[cluster_level] == cluster_id]['occupation']).most_common(4)
    top5_dict[cluster_id] = top5

top5_dict

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 30, Finished, Available)

{'B01': [('Bankers', 11), ('Doctors', 11), ('Salespeople', 7), ('CEOs', 6)],
 'B02': [('Construction Workers', 15),
  ('Police Officers', 6),
  ('Bankers', 4),
  ('Artists', 4)],
 'B03': [('CEOs', 8), ('Actors', 8), ('Bankers', 3), ('Politicians', 2)],
 'B04': [('Bankers', 9),
  ('Doctors', 7),
  ('Actors', 5),
  ('Police Officers', 4)],
 'B05': [('Bankers', 22), ('Lawyers', 5), ('Artists', 5), ('Politicians', 5)],
 'B06': [('Programmers', 10),
  ('Engineers', 4),
  ('Accountants', 2),
  ('Biologists', 2)],
 'B07': [('Engineers', 6),
  ('Programmers', 4),
  ('Waiters', 3),
  ('Physicists', 2)],
 'B08': [('Artists', 9), ('Actors', 5), ('Doctors', 3), ('Accountants', 3)],
 'B09': [('Bankers', 15), ('Accountants', 9), ('CEOs', 8), ('Artists', 8)],
 'B10': [('Engineers', 9),
  ('Accountants', 9),
  ('Programmers', 7),
  ('Physicists', 2)],
 'B11': [('Bankers', 6), ('Actors', 5), ('Chefs', 3), ('Hairdressers', 3)],
 'B12': [('Chefs', 12),
  ('Construction Workers', 11),
  ('Mechanics', 8),


In [19]:
# Counter example: not all clusters make sense
cluster_id = 'D091'
generic_occupation_stereotype[generic_occupation_stereotype[cluster_level] == cluster_id]['generic'].values

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 31, Finished, Available)

array([], dtype=object)

In [20]:
honest_flag = [ 'honest' in sentence for sentence in generic_occupation_stereotype['generic']]
generic_occupation_stereotype[ honest_flag ]

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 32, Finished, Available)

,stereotype,generic,occupation,embedding,cluster_A,cluster_B,cluster_C,cluster_D,cluster_E
1900,Bankers are greedy and dishonest.,They are greedy and dishonest.,Bankers,"[-0.10204800218343735, -0.021863102912902832, ...",A01,B05,C013,D028,E0056
264,Waiters are not honest.,They are not honest.,Waiters,"[0.35267525911331177, -0.13989655673503876, -0...",A04,B20,C062,D165,E0317
3266,Retail Workers are not very honest.,They are not very honest.,Retail Workers,"[0.3177037835121155, -0.1410876214504242, -0.3...",A04,B20,C062,D165,E0317
143,Lawyers are never honest.,They are never honest.,Lawyers,"[0.251641184091568, 0.14636151492595673, -0.65...",A04,B20,C062,D165,E0318
130,All lawyers are dishonest.,They are dishonest.,Lawyers,"[0.012324146926403046, -0.5023002624511719, -0...",A07,B27,C091,D237,E0439
154,Mechanics are dishonest.,They are dishonest.,Mechanics,"[0.012324146926403046, -0.5023002624511719, -0...",A07,B27,C091,D237,E0439
488,Lawyers are dishonest.,They are dishonest.,Lawyers,"[0.012324146926403046, -0.5023002624511719, -0...",A07,B27,C091,D237,E0439
567,Physicists are all dishonest.,They are all dishonest.,Physicists,"[-0.3060470521450043, -0.4607492983341217, -0....",A07,B27,C091,D237,E0439
630,All waiters are dishonest.,They are dishonest.,Waiters,"[0.012324146926403046, -0.5023002624511719, -0...",A07,B27,C091,D237,E0439
784,All lawyers are dishonest.,They are dishonest.,Lawyers,"[0.012324146926403046, -0.5023002624511719, -0...",A07,B27,C091,D237,E0439


## Find candidate cluster names using TF/IDF

In [21]:
def get_candidate_names(cluster_text_pdf, cluster_col, text_col, 
                        sentence_sep=' . ', max_ngram_length=3, top_n=1):
    """
    Use TF-IDF to find terms to use as candidate names for sections of text (especially clusters). 
    This function returns the cluster candidate names for each member of the cluster; use 'get_cluster_titles' 
    to summarize this by cluster.

    Example:
    cluster_sentence_pdf = article_sentence.copy()
    cluster_sentence_pdf['cluster_A_name'] = get_candidate_names(cluster_sentence_pdf, cluster_col='cluster_A', text_col='sentence')
    """
    import numpy as np
    from sklearn.feature_extraction.text import TfidfVectorizer


    def combine_sentences(s):
        return sentence_sep.join(s)

    corpus = cluster_text_pdf[[cluster_col, text_col]]\
                    .groupby([cluster_col])[text_col]\
                    .transform(combine_sentences)\
                    .values

    corpus = np.array([s.replace("don't", "do not")\
                       .replace("do not ", "do_not_")\
                       .replace(' not ', ' not_')\
                       .replace('very ', 'very_')\
                       .replace('at ', 'at_')\
                       .replace('good at', 'good_at')
                       for s in corpus])

    stop_words = ['they', 'are', 'always', sentence_sep.strip()]
    tfidf = TfidfVectorizer(ngram_range=(1, max_ngram_length), 
                            stop_words=stop_words, # sublinear_tf=True, # max_df=0.5,
                            lowercase=True)
    X = tfidf.fit_transform(corpus)
    feature_names = tfidf.get_feature_names_out()

    # give slight advantage or disadvantage to terms containing more words
    npX = np.array(X.todense())
    smidge = 1e-6  # just enough to break ties, not enough to affect real differences

    feature_tfidf_adjustment = [smidge * len(n.split(' ')) for n in feature_names]
    adjusted_X = np.array([npX[i,:] + feature_tfidf_adjustment for i in range(len(npX))])
    # candidate_names = [x for x in np.array(feature_names)[adjusted_X.argmax(axis=1)].tolist()]

    top_idx = [ [i for i in v[-top_n:][::-1]] for v in adjusted_X.argsort(axis=1)] # [-n:]
    candidate_names = [', '.join(x) for x in np.array(feature_names)[top_idx].tolist()]

    non_repetitive_candidate_names = [re.sub(r"(([^ ]{2,12})+) \1", r"\1", candidate_name) for candidate_name in candidate_names]
    adjusted_tfidf = adjusted_X.max(axis=1)

    return non_repetitive_candidate_names #, adjusted_tfidf

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 33, Finished, Available)

In [22]:
generic_occupation_stereotype['tag'] = get_candidate_names(generic_occupation_stereotype, cluster_level, # 'cluster_C', 
                                            text_col = 'generic', max_ngram_length=1, 
                                            sentence_sep='. ')

tag_counts = Counter(generic_occupation_stereotype['tag'])

common_tags = [tag for (tag, count) in tag_counts.most_common() if count >= 10]

len(common_tags) # 131

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 34, Finished, Available)

67

In [23]:
is_common = [tag in common_tags for tag in generic_occupation_stereotype['tag']]
occupation_tag = generic_occupation_stereotype[ is_common ][ ['occupation', 'tag']]


basket_item2 = pd.melt(occupation_tag.reset_index().rename(columns={'index':'basket'}), 
                        id_vars='basket', 
                        value_vars=['occupation', 'tag'],
                        var_name='category', value_name='item'
                     ).sort_values(['basket', 'category'])

basket_item2

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 35, Finished, Available)

,basket,category,item
3308,0,occupation,CEOs
6666,0,tag,ways
959,1,occupation,CEOs
4317,1,tag,only
2246,2,occupation,CEOs
...,...,...,...
6385,3355,tag,angry
3019,3356,occupation,Waiters
6377,3356,tag,angry
2940,3357,occupation,Waiters


In [24]:
item_pair_stats2 = get_item_pair_stats(basket_item2)

item_pair_stats2

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 36, Finished, Available)

,item1,item2,item1_count,item2_count,both_count,item1_prevalence,item2_prevalence,confidence,lift
0,Accountants,about,141,96,1,0.041989,0.028588,0.007092,0.248079
1,Accountants,aggressive,141,46,2,0.041989,0.013699,0.014184,1.035461
2,Accountants,boring,141,29,14,0.041989,0.008636,0.099291,11.497188
3,Accountants,calm,141,19,1,0.041989,0.005658,0.007092,1.253453
4,Accountants,charming,141,61,1,0.041989,0.018166,0.007092,0.390420
...,...,...,...,...,...,...,...,...,...
1701,young,Police Officers,74,40,2,0.022037,0.011912,0.027027,2.268919
1702,young,Programmers,74,114,2,0.022037,0.033949,0.027027,0.796112
1703,young,Retail Workers,74,101,1,0.022037,0.030077,0.013514,0.449291
1704,young,Salespeople,74,100,1,0.022037,0.029780,0.013514,0.453784


In [25]:
nodes2, edges2 = get_nodes_and_edges_from_item_pair_stats( item_pair_stats2[ item_pair_stats2['both_count'] > 5 ] )


StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 37, Finished, Available)

Your graph will have 92 nodes and 362 edges.


In [26]:
occupations2 = set(generic_occupation_stereotype['occupation'])

nodes2['category'] = [ 'occupation' if x in occupations2 else 'tag' for x in nodes2['label'] ]
nodes2['shape'] = [ 'box' if cat == 'occupation' else 'ellipse' for cat in nodes2['category'] ]
nodes2['color'] = [ '#FF9999' if cat == 'occupation' else '#9999FF' for cat in nodes2['category'] ]

def make_node_title(row, tag_col, sentence_df, text_col):
    title = f"{row['label']}\n({row['category']}, {row['count']} examples)"
    if row['category'] == 'tag':
        tag = row['label']
        examples = sentence_df[ sentence_df[tag_col] == tag ][text_col].values
        top_examples = [ f"{x[0]} ({x[1]} times)" for x in Counter(examples).most_common(5) ]
        title += '\n' + '\n'.join(top_examples)
        
    return title

nodes2['title'] = [ make_node_title(row, "tag", generic_occupation_stereotype, 'generic') for row in nodes2.to_dict(orient='records') ]

nodes2

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 38, Finished, Available)

,label,count,prevalence,id,category,shape,color,title
0,Accountants,141,0.041989,0,occupation,box,#FF9999,"Accountants\n(occupation, 141 examples)"
1,Actors,125,0.037225,1,occupation,box,#FF9999,"Actors\n(occupation, 125 examples)"
2,Architects,52,0.015485,2,occupation,box,#FF9999,"Architects\n(occupation, 52 examples)"
3,Artists,140,0.041691,3,occupation,box,#FF9999,"Artists\n(occupation, 140 examples)"
4,Bankers,281,0.083681,4,occupation,box,#FF9999,"Bankers\n(occupation, 281 examples)"
...,...,...,...,...,...,...,...,...
87,uneducated,69,0.020548,87,tag,ellipse,#9999FF,"uneducated\n(tag, 69 examples)\nThey are unedu..."
88,untrustworthy,77,0.022930,88,tag,ellipse,#9999FF,"untrustworthy\n(tag, 77 examples)\nThey are un..."
89,ways,50,0.014890,89,tag,ellipse,#9999FF,"ways\n(tag, 50 examples)\nThey are always look..."
90,working,68,0.020250,90,tag,ellipse,#9999FF,"working\n(tag, 68 examples)\nThey are always w..."


In [27]:
edges2['weight'] = edges2['confidence']

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 39, Finished, Available)

In [28]:
vis_js_html = get_vis_js_html(nodes2, edges2)
displayHTML(vis_js_html)

StatementMeta(, 366a20af-8b9a-4631-b4cf-9dabeadcd56e, 40, Finished, Available)